In [7]:
import sys
import pandas as pd

sys.path.append("..")

import numpy as np
import pandas as pd
from xai_ranking.benchmarks import (
    human_in_the_loop,
    hierarchical_ranking_explanation,
    lime_experiment,
    shap_experiment,
    sharp_experiment,
    participation_experiment,
)
from xai_ranking.preprocessing import (
    preprocess_atp_data,
    preprocess_csrank_data,
    preprocess_higher_education_data,
)
from xai_ranking.datasets import (
    fetch_atp_data,
    fetch_csrank_data,
    fetch_higher_education_data,
    fetch_movers_data,
)
from xai_ranking.scorers import (
    atp_score,
    csrank_score,
    higher_education_score,
)

RNG_SEED = 42

In [3]:
datasets = [
    {
        "name": "CSRank",
        "data": fetch_csrank_data(),
        "preprocess": preprocess_csrank_data,
        "scorer": csrank_score,
    },
]
xai_methods = [
    {"name": "LIME", "experiment": lime_experiment},
    {"name": "SHAP", "experiment": shap_experiment},
    {"name": "ShaRP", "experiment": sharp_experiment},
    # {"name": "Participation", "experiment": participation_experiment},
    {"name": "HRE", "experiment": hierarchical_ranking_explanation},
    {"name": "HIL", "experiment": human_in_the_loop},
]

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


In [25]:
results = {}
for dataset in datasets:
    results[dataset["name"]] = {}
    for xai_method in xai_methods:
        experiment_func = xai_method["experiment"]
        preprocess_func = dataset["preprocess"]
        score_func = dataset["scorer"]
        X, ranks, scores = preprocess_func(dataset["data"])
        contributions = experiment_func(X, score_func)
        results[dataset["name"]][xai_method["name"]] = contributions
        
        result_df = pd.DataFrame(contributions, columns=X.columns, index=X.index)
        result_df.to_csv(f"results/_contributions_{dataset['name']}_{xai_method['name']}.csv")
        # with open(f"_contributions_{dataset['name']}_{xai_method['name']}.npy", "wb") as f:
        #     np.save(f, contributions)

In [5]:
results

{'ATP': {'LIME': array([[ 3.12376571,  2.00759403, -3.70052477,  1.22817005,  8.00124511,
          -0.53089756],
         [ 2.81473197, -0.28754179,  1.35617903,  2.53889913, -0.36674666,
          -1.21787992],
         [-0.78240949,  0.47546936,  3.86544551,  0.17699443, -0.02957212,
           1.02093902],
         [-3.60789384, -1.50606483,  0.69376593, -2.58814814, -5.3129342 ,
           1.15970308],
         [-3.54545993, -1.46871099, -3.70560436, -2.58181931, -5.19558699,
           1.06457435]]),
  'SHAP': array([[ 2.6 ,  1.6 , -1.32,  0.96,  6.42, -0.32],
         [ 2.2 , -0.3 ,  1.08,  2.16, -0.08, -1.02],
         [-0.6 ,  0.4 ,  2.98,  0.06, -0.08,  0.28],
         [-2.4 , -0.9 ,  0.48, -0.24, -2.68,  0.78],
         [-1.8 , -0.8 , -3.22, -2.94, -3.58,  0.28]]),
  'ShaRP': array([[ 0.44666667,  0.43666667, -0.37      ,  0.14      ,  1.42666667,
          -0.08      ],
         [ 0.49666667, -0.08333333,  0.26666667,  0.64333333, -0.02666667,
          -0.29666667],
      